In [1]:
import os
import requests
import zipfile

# URL del dataset comprimido
url = "https://vision.eng.au.dk/?download=/data/WeedData/Nonsegmented.zip"

# Directorio de destino para guardar los archivos descargados
target_dir = "/content/dataset"

# Crear el directorio de destino si no existe
os.makedirs(target_dir, exist_ok=True)

# Nombre del archivo comprimido
zip_filename = os.path.join(target_dir, "dataset.zip")

# Descargar el archivo comprimido desde la URL
response = requests.get(url)
with open(zip_filename, "wb") as f:
    f.write(response.content)

# Descomprimir el archivo
with zipfile.ZipFile(zip_filename, "r") as zip_ref:
    zip_ref.extractall(target_dir)

# Directorio de datos de entrenamiento
train_dir = os.path.join(target_dir, "Nonsegmented")

In [11]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, LearningRateScheduler

# Configurar generador de imágenes para el entrenamiento y validación
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=0.1,
    validation_split=0.2
)

# Generar datos de entrenamiento desde el directorio
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Generar datos de validación desde el directorio
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Crear el modelo EfficientNetB4
base_model = EfficientNetB4(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = False  # Congelar las capas del modelo base

# Agregar capas al modelo
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(12, activation='softmax'))

# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Configurar callbacks
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# Callback para ajuste de tasa de aprendizaje
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

# Entrenar el modelo
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=5,  # Sin aumentar las épocas
    callbacks=[checkpoint, reduce_lr, early_stopping, lr_scheduler]
)

# Descongelar las capas superiores del modelo base y recompilar
base_model.trainable = True

# Congelar capas específicas
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Recompilar el modelo con una tasa de aprendizaje más baja
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continuar entrenando el modelo
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=5,  # Sin aumentar las épocas totales
    callbacks=[checkpoint, reduce_lr, early_stopping, lr_scheduler]
)


Found 4440 images belonging to 12 classes.
Found 1104 images belonging to 12 classes.
Epoch 1/5
139/139 [==============================] - ETA: 0s - loss: 9.2256 - accuracy: 0.0910
Epoch 1: val_loss improved from inf to 6.00604, saving model to best_model.h5
139/139 [==============================] - 150s 961ms/step - loss: 9.2256 - accuracy: 0.0910 - val_loss: 6.0060 - val_accuracy: 0.1377 - lr: 0.0010
Epoch 2/5
139/139 [==============================] - ETA: 0s - loss: 5.5988 - accuracy: 0.1043
Epoch 2: val_loss improved from 6.00604 to 4.42785, saving model to best_model.h5
139/139 [==============================] - 126s 906ms/step - loss: 5.5988 - accuracy: 0.1043 - val_loss: 4.4279 - val_accuracy: 0.1042 - lr: 0.0010
Epoch 3/5
139/139 [==============================] - ETA: 0s - loss: 4.3292 - accuracy: 0.1041
Epoch 3: val_loss improved from 4.42785 to 3.75910, saving model to best_model.h5
139/139 [==============================] - 127s 915ms/step - loss: 4.3292 - accuracy: 0.104

In [12]:
import os
from google.colab import drive
import shutil

# Montar Google Drive
drive.mount('/content/drive')

# Directorio donde está la carpeta "test" en Google Drive
drive_dir = '/content/drive/My Drive/test'

# Directorio de destino para extraer las imágenes
target_dir = '/content/dataset/test'

# Crear el directorio de destino si no existe
os.makedirs(target_dir, exist_ok=True)

# Lista de archivos en la carpeta "test"
file_list = os.listdir(drive_dir)

# Copiar cada archivo de la carpeta "test" a la carpeta de destino
for file in file_list:
    file_path = os.path.join(drive_dir, file)
    if os.path.isfile(file_path):
        shutil.copy(file_path, target_dir)

print("Imágenes copiadas correctamente.")


import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.applications.resnet50 import preprocess_input

# Directorio donde se encuentran las imágenes de prueba
test_dir = "/content/dataset/test"

# Obtener la lista de nombres de archivo de las imágenes de prueba
test_filenames = os.listdir(test_dir)

# Preprocesar las imágenes y cargarlas en un arreglo numpy
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)  # Preprocesar la imagen para que coincida con el preprocesamiento del modelo
    test_images.append(img_array)

# Convertir la lista de imágenes en un arreglo numpy
test_images = np.array(test_images)

# Clasificar las imágenes utilizando el modelo
predictions = model.predict(test_images)

# Obtener las categorías predichas para cada imagen
predicted_categories = np.argmax(predictions, axis=1)

# Obtener los nombres de las categorías predichas
class_names = train_generator.class_indices.keys()
predicted_labels = [list(class_names)[i] for i in predicted_categories]

# Crear un DataFrame para almacenar los resultados
results_df = pd.DataFrame({"file": test_filenames, "species": predicted_labels})

# Guardar el DataFrame como un archivo CSV
results_df.to_csv("/content/predictions.csv", index=False)

from google.colab import files

files.download("/content/predictions.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Imágenes copiadas correctamente.
33/33 [==============================] - 9s 194ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>